# Importing Necessary Libraries

In [95]:
#Importing libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
df = pd.read_csv("/content/drive/MyDrive/spam_detection/processed_data.csv", encoding = 'utf-8')
df.head()

,label,subject,email_to,email_from,message
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...


In [98]:
df.tail()

,label,subject,email_to,email_from,message
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"For those who are interested, I just cook a li..."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."
75418,1,I wanted the desk at his own laws: of the. Bu...,the00@plg.uwaterloo.ca,"""Danny"" <pwcusnt@noblecoffee.com>",Content-Type: multipart/alternative;\n\tbounda...


In [99]:
for i in range(10):
    print("Message: " + df.loc[i]['message'])
    print("-----------------------------------------------------------------------------------")

Message: Content-Type: text/html;
Content-Transfer-Encoding: 7Bit Do you feel the pressure to perform and not rising to the occasion?? Try V ia gr a ..... your anxiety will be a thing of the past and you will be back to your old self.
-----------------------------------------------------------------------------------
Message: Hi, i've just updated from the gulus and I check on other mirrors.
It seems there is a little typo in /debian/README file

Example:
http://gulus.usherbrooke.ca/debian/README
ftp://ftp.fr.debian.org/debian/README

"Testing, or lenny.  Access this release through dists/testing.  The
current tested development snapshot is named etch.  Packages which
have been tested in unstable and passed automated tests propogate to
this release."

etch should be replace by lenny like in the README.html



-- 
Yan Morin
Consultant en logiciel libre
yan.morin@savoirfairelinux.com
514-994-1556


-- 
To UNSUBSCRIBE, email to debian-mirrors-REQUEST@lists.debian.org
with a subject of "un

# Preprocessing

In [100]:
#checking for missing values
df.isnull().sum()

label            0
subject        793
email_to       576
email_from       0
message       1487
dtype: int64

In [101]:
df.dropna(inplace=True)
print(df.isnull().sum())

label         0
subject       0
email_to      0
email_from    0
message       0
dtype: int64


In [102]:
df.groupby("label").count()

,subject,email_to,email_from,message
label,,,,
0,25192,25192,25192,25192
1,47606,47606,47606,47606


In [103]:
#renaming columns
df.rename(columns={'subject': 'sub', 'email_to': 'to', 'email_from': 'from', 'message': 'msg'}, inplace=True)
sd = df
df.head()

,label,sub,to,from,msg
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...


In [104]:
sd.drop(['to'], axis = 1, inplace = True)   
sd.head()

,label,sub,from,msg
0,1,"Generic Cialis, branded quality@","""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...


In [105]:
#Lower casing the strings
sd['msg'] = sd['msg'].str.lower()
sd['sub'] = sd['sub'].str.lower()
sd['from'] = sd['from'].str.lower()
sd.head()

,label,sub,from,msg
0,1,"generic cialis, branded quality@","""tomas jacobs"" <rickyames@aol.com>",content-type: text/html;\ncontent-transfer-enc...
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,"hi, i've just updated from the gulus and i che..."
2,1,authentic viagra,"""sheila crenshaw"" <7stocknews@tractionmarketin...","content-type: text/plain;\n\tcharset=""iso-8859..."
3,1,nice talking with ya,"""stormy dempsey"" <vqucsmdfgvsg@ruraltek.com>","hey billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,"""christi t. jernigan"" <dcube@totalink.net>",content-type: multipart/alternative;\n ...


In [106]:
#Removing punctuations and numbers from 'msg' column

sd['msg'] = sd['msg'].apply(lambda x:re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", '', x))
sd['msg'] = sd['msg'].apply(lambda x: re.sub(r'\b\w{10,}\b', '', x))
sd.msg = sd.msg.replace(r'\s+', ' ', regex=True)
sd['msg'] = df['msg'].str.replace('textplain', '')

for i in range(10):
    print("Message: " + df.loc[i]['msg'])
    print("-----------------------------------------------------------------------------------")

Message:  7bit do you feel the pressure to perform and not rising to the occasion try v ia gr a your anxiety will be a thing of the past and you will be back to your old self
-----------------------------------------------------------------------------------
Message: hi ive just updated from the gulus and i check on other mirrorsit seems there is a little typo in or lenny access this release through tested snapshot is named etch packages whichhave been tested in unstable and passed automated tests propogate tothis should be replace by lenny like in the yan en logiciel to email to a subject of trouble contact 
-----------------------------------------------------------------------------------
Message:   7bitmega i a g r a discount pricec i a l i s discount pricedo not miss it click texthtml 7bit authentic viagra mega authentic v i a g r a discount price c i a l i s discount price do not miss it click here
----------------------------------------------------------------------------------

In [107]:
sd.head()

,label,sub,from,msg
0,1,"generic cialis, branded quality@","""tomas jacobs"" <rickyames@aol.com>",7bit do you feel the pressure to perform and ...
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,hi ive just updated from the gulus and i check...
2,1,authentic viagra,"""sheila crenshaw"" <7stocknews@tractionmarketin...",7bitmega i a g r a discount pricec i a l i s...
3,1,nice talking with ya,"""stormy dempsey"" <vqucsmdfgvsg@ruraltek.com>",hey billy it was really fun going out the othe...
4,1,or trembling; stomach cramps; trouble in sleep...,"""christi t. jernigan"" <dcube@totalink.net>",of the home it will have the to be linked fa...


In [108]:
# Importing 
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [109]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def rmv_stopwords(string):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    filtered_tokens = [w for w in tokens if not w in stop_words]
    filtered_sentence = ' '.join(filtered_tokens)
    return filtered_sentence


lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

sd['msg'] = sd['msg'].apply(rmv_stopwords)
sd['msg'] = sd['msg'].apply(lemmatize_words)
sd[sd['msg'].map(lambda x: x.isascii())]

sd.head()

,label,sub,from,msg
0,1,"generic cialis, branded quality@","""tomas jacobs"" <rickyames@aol.com>",7bit feel pressure perform rise occasion try v...
1,0,typo in /debian/readme,yan morin <yan.morin@savoirfairelinux.com>,hi ive update gulus check mirrorsit seem littl...
2,1,authentic viagra,"""sheila crenshaw"" <7stocknews@tractionmarketin...",7bitmega g r discount pricec l discount priced...
3,1,nice talking with ya,"""stormy dempsey"" <vqucsmdfgvsg@ruraltek.com>",hey billy really fun go night talk say manhood...
4,1,or trembling; stomach cramps; trouble in sleep...,"""christi t. jernigan"" <dcube@totalink.net>",home link far know within part respect affect ...


# Model Building

In [110]:
string = sd['msg']
labels = sd['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(string, labels, test_size=0.3, random_state=58)

In [111]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
def perform(y_pred):
    print("Precision : ", precision_score(y_test, y_pred))
    print("Recall : ", recall_score(y_test, y_pred))
    print("Accuracy Score : ", accuracy_score(y_test, y_pred))
    print("F1 Score : ", f1_score(y_test, y_pred))
    print("\n", confusion_matrix(y_test, y_pred))
    print("")

In [112]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
  
vectorization = TfidfVectorizer()
X_train_vectorized = vectorization.fit_transform(X_train)
X_test_vectorized = vectorization.transform(X_test)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_vectorized, y_train)

y_pred = naive_bayes.predict(X_test_vectorized)

perform(y_pred)

Precision :  0.9779852305977428
Recall :  0.9831220673716646
Accuracy Score :  0.974496336996337
F1 Score :  0.9805469213844165

 [[ 7245   316]
 [  241 14038]]



# Testing

In [113]:
not_spam = [
"Hello team, This is a reminder that we have a team meeting scheduled for tomorrow at 10 AM in the conference room. Please come prepared with your updates and any important topics you'd like to discuss. Looking forward to seeing you all there.",
"Dear Customer, Thank you for your recent purchase. We are pleased to inform you that your payment has been successfully processed, and your order is now being prepared for shipment. We will notify you once it's dispatched. If you have any questions, feel free to reach out to our customer support. Thank you for shopping with us!",
"Dear Max, You are cordially invited to our Annual Charity Gala on 14th May, 2023 at Steve Jobs Hall. Join us for an evening of celebration and fundraising for a great cause. Please RSVP by 14th August, 2023 to secure your seat. We look forward to your presence at this special event.",
"Dear Employees, We would like to inform you about some important updates to our company policies. These changes aim to improve efficiency and maintain a positive work environment. Please take a moment to review the updated policies attached to this email. If you have any questions or need clarification, don't hesitate to reach out to the HR department. Thank you for your attention to this matter.",
"Hello Hamilton, Just a friendly reminder about the upcoming webinar on digital marketing strategies, hosted by our industry expert, Toto Wolff. The webinar will take place on 2nd May, 2023 at 9:00PM. Don't miss this opportunity to gain valuable insights and stay ahead in the digital marketing landscape. Register now to secure your spot!",
"Dear Ahmed, We would like to extend our warmest congratulations on your work anniversary with our company. Your dedication and hard work have been instrumental in our success. Thank you for your contributions, and we look forward to many more years of working together. Cheers!",
"Dear Subscriber, We are pleased to announce the publication of a new article on our website: 'Tips for Effective Time Management.' This informative piece provides valuable insights and practical strategies to optimize your time and enhance productivity. Click the link below to read the article and let us know your thoughts. Happy reading!",
"Hello Team, Just a quick reminder about the training session on the new software system scheduled for 22nd May, 2022 at 3:30PM. This training is mandatory for all team members to ensure a smooth transition and efficient utilization of the new system. Please come prepared with your questions and enthusiasm to learn. See you there!",
"Dear Community Members, We are organizing a beach cleanup event on 27th of February at Miami Beach. Join us in making a positive impact on our environment and preserving the beauty of our beaches. Bring your friends and family for a day of fun and community service. Together, we can make a difference. Sign up now to secure your spot!",
"Dear Sebastian, On behalf of Mercedes Petronas AMG, we would like to express our sincere gratitude for your generous donation. Your support will directly impact the lives of those in need and contribute to our ongoing efforts. We are immensely grateful for your kindness and belief in our mission. Thank you once again for your contribution."
]

spam = [
    "Dear winner, you have been selected as the lucky recipient of an all-expenses-paid luxury vacation. Click here to claim your prize!",
    "Hello, I have discovered a secret method to make thousands of dollars in just a few days. Don't miss out on this incredible opportunity. Act now!",
    "Dear user, it has come to our attention that your account has been compromised. Click the link below to secure your account immediately.",
    "Good day to you, Money was donated to you by Mr Pedro. Please reply to redeem. Thank you.",
    "Greetings, I am a Nigerian prince seeking assistance in transferring a large sum of money. Help me, and you will be generously rewarded. Reply now!",
    "Dear student, tired of studying? Obtain a degree from a prestigious university without any exams or coursework. Order your diploma now!",
    "Struggling to shed those extra pounds? Our amazing weight loss supplement guarantees quick and effortless results. Try it today and see the difference!",
    "Congratulations! You are the lucky winner of a huge lottery jackpot. To claim your prize, provide your personal details and pay a small processing fee.",
    "Looking for love? We have a database of attractive singles in your vicinity. Sign up now and start connecting with them instantly!",
    "Dear valued customer, don't miss this exclusive opportunity to enjoy massive discounts on our premium products. Click here to unlock your savings!"
]

In [114]:
# test_string = [spam[]]
test_string_vectorized = vectorization.transform(test_string)
prediction = naive_bayes.predict(test_string_vectorized)
print("Prediction:", prediction)

Prediction: [0]


Using real life emails

In [126]:
test_string = ["Hi John, Someone just used your password to try to sign in to your Google Account john.podesta@gmail.com. Details: Saturday 19 March, 8:34:40 UTC IP Address: 134.249.139.239 Locatiom: Ukraine. Google stopped this sign-in attempt. You shoud change your password immediately. Change Password Best, The Gmail Team"]
test_string_vectorized = vectorization.transform(test_string)
prediction = naive_bayes.predict(test_string_vectorized)
print("Prediction:", prediction)

Prediction: [0]


Spam:

- NB: 1/4 

**Actual Spam Mails:**

- You've received a pdf document from OneDrive Cloud. Number of images: 1. Attachment File Type: PDF. Review Awaiting Files. For more information on OneDrive products and solutions, please vist OneDrive

- You have received a payment from the ATO of $6301.07 with a reference number Direct Credit 012721 ATO ATO003000010459010. Please confirm. Click to view, Remittance_Advise_012721.pdf IMPORTANT The information transmitted is for the use of the intended recipient only and may contain confidential and/or legally privileged material. Any review, re-transmission, disclosure, dissemination or other use of, or taking of any action in reliance upon, this information by persons or entities other than the intended recipient is prohibited and may result in severe penalties. If you have received this e-mail in errorplease notify the Privacy Hotline of the Australian Taxation Office, telephone 1300 661 542 and delete all copies of this transmission together with any attachments. *** [Got correct]

- We've preveneted the delivery of 6 new incoming emails to your inbox as of 2/25/2019 8:00:00 AM Recover_Messages Microsoft

- Hi John, Someone just used your password to try to sign in to your Google Account john.podesta@gmail.com. Details: Saturday 19 March, 8:34:40 UTC IP Address: 134.249.139.239 Locatiom: Ukraine. Google stopped this sign-in attempt. You shoud change your password immediately. Change Password Best, The Gmail Team


In [118]:
test_string = ["Dear Abdullah, Kindly look into the reports that were sent by you for review. It contains certain levels of plagiarism and it's better to minimize it. Looking forward for the updated version of it."]
test_string_vectorized = vectorization.transform(test_string)
prediction = naive_bayes.predict(test_string_vectorized)
print("Prediction:", prediction)

Prediction: [0]


Actual Non-spam mails:

- NB: 2/4

- Dear Students, You are advised to note that the lab exam of English-II (SL1006) will be held on Tuesday May 23, 2023 at 04:00 PM in the same room where the subjective exam will be conducted. Do not leave the room after finishing the subjective exam. Regards, Waqas Javed Ahmad Central Academic Office FAST-NATIONAL UNIVERSITY of Computer & Emerging Sciences, Lahore Campus 042-111-128 -128 (Ext: 289) PRIVILEGED AND CONFIDENTIAL. This e-mail and any attachments may contain confidential and privileged information for the sole use of the addressee(s) named above. Any review, use, copying, distribution or disclosure by others is strictly prohibited. If you have received this e-mail in error, please notify the sender by replying to the e-mail and then immediately delete the original message and copies made there from. Thank you. 

- Microsoft account Password reset code Please use this code to reset the password for the Microsoft account ib*****@outlook.com. Here is your code: 0474594 If you don't recognize the Microsoft account ib*****@outlook.com, you can click here to remove your email address from that account. Thanks, The Microsoft account team [Incorrect]

- Dear Abdullah, Good to hear from you and your team. I will be available from 22nd August. Once the office space is allocated to me, we will be able to meet. Please email me on Monday evening, I will share the time of meeting with you for Tuesday, 23rd. Dr. Mohsin Kamal [Incorrect]

- Dear Abdullah, Kindly look into the reports that were sent by you for review. It contains certain levels of plagiarism and it's better to minimize it. Looking forward for the updated version of it.



In [125]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

test_string = ["We've preveneted the delivery of 6 new incoming emails to your inbox as of 2/25/2019 8:00:00 AM Click On Recover_Messages Microsoft"]

# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# def rmv_stopwords(string):
#     tokens = word_tokenize(string)
#     filtered_tokens = [w for w in tokens if not w in stop_words]
#     filtered_sentence = ' '.join(filtered_tokens)
#     return filtered_sentence

# def lemmatize_words(text):
#     words = text.split()
#     words = [lemmatizer.lemmatize(word, pos='v') for word in words]
#     return ' '.join(words)

# processed_strings = [lemmatize_words(rmv_stopwords(s)) for s in test_string]
test_string_vectorized = vectorization.transform(test_string)
prediction = naive_bayes.predict(test_string_vectorized)

print("Prediction:", prediction)

Prediction: [0]
